In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM, T5Tokenizer, T5ForConditionalGeneration, LlamaTokenizer, LlamaForCausalLM, GenerationConfig
import torch
from peft import PeftModel

device = "cuda:0" if torch.cuda.is_available() else "cpu"
# tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xl")
# model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xl")

tokenizer = LlamaTokenizer.from_pretrained("decapoda-research/llama-7b-hf")

BASE_MODEL = "decapoda-research/llama-7b-hf"
LORA_WEIGHTS = "tloen/alpaca-lora-7b"

model = LlamaForCausalLM.from_pretrained(
        BASE_MODEL,
        load_in_8bit=False,
        torch_dtype=torch.float16,
        device_map="auto",
)
model = PeftModel.from_pretrained(
    model, LORA_WEIGHTS, torch_dtype=torch.float16, force_download=True
)

c:\Users\Aviv9\.virtualenvs\Robo-GPT-QaSuWWH8\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Aviv9\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result 

In [5]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [9]:
torch.cuda.empty_cache()

In [8]:
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

print(f"{device=}")
pipe = pipeline(
    "text2text-generation",
    model=model, 
    tokenizer=tokenizer, 
    max_length=100,
    device=0
)

llm = LLMChain(prompt=prompt, llm=HuggingFacePipeline(pipeline=pipe))

device='cuda:0'


OutOfMemoryError: CUDA out of memory. Tried to allocate 86.00 MiB (GPU 0; 10.00 GiB total capacity; 9.22 GiB already allocated; 0 bytes free; 9.22 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
general_directions = """
Your decisions must always be made independently without seeking user assistance. Play to your strengths as an LLM and pursue simple strategies with no legal complications.


CONSTRAINTS:
1. No user assistance.
2. Cannot run Python code that requires user input.


ACTIONS:

1. "READ_FILE": read the current state of a file. The schema for the action is:

READ_FILE: <PATH>

2. "WRITE_FILE": write a block of text to a file. The schema for the action is:

WRITE_FILE: <PATH>
```
<TEXT>
```

3. "RUN_PYTHON": run a Python file. The schema for the action is:

RUN_PYTHON: <PATH>

4. "SEARCH_ONLINE": search online and get back a list of URLs relevant to the query. The schema for the action is:

SEARCH_ONLINE: <QUERY>

5. EXTRACT_INFO: extract specific information from a webpage. The schema for the action is:

EXTRACT_INFO: <URL>, <a brief instruction to GPT for information to extract>

6. "SHUTDOWN": shut down the program. The schema for the action is:

SHUTDOWN


RESOURCES:
1. File contents after reading file.
2. Online search results returning URLs.
3. Output of running a Python file.


PERFORMANCE EVALUATION:
1. Continuously review and analyze your actions to ensure you are performing to the best of your abilities. 
2. Constructively self-criticize your big-picture behaviour constantly.
3. Reflect on past decisions and strategies to refine your approach.
4. Every action has a cost, so be smart and efficent. Aim to complete tasks in the least number of steps.


Write only one action. The action must one of the actions specified above and must be written according to the schema specified above.

After the action, also write the following metadata JSON object, which must be parsable by Python's json.loads():
/{
    "criticism": "<constructive self-criticism of actions performed so far, if any>",
    "reason": "<a sentence explaining the action above>",
    "plan": "<a short high-level plan in plain English>",
    "speak": "<a short summary of thoughts to say to the user>"
/}

If you want to run an action that is not in the above list of actions, send the SHUTDOWN action instead and explain which action you wanted to run in the metadata JSON object.
So, write one action and one metadata JSON object, nothing else.
Goal: {goal}
AI:
"""

In [ ]:
robo_agent = LLMChain(prompt=PromptTemplate(template=general_directions, input_variables=["goal"]), llm=HuggingFacePipeline(pipeline=pipe))

ValidationError: 1 validation error for PromptTemplate
__root__
  Invalid prompt schema; check for mismatched or missing input parameters. '\n    "criticism"' (type=value_error)

In [ ]:
robo_agent.run("I want you to increase my profile searches in linkedin")

NameError: name 'robo_agent' is not defined